# Autograd: 自动求导

**PyTorch中，所有神经网络的核心是 autograd 包。autograd 包为张量上的所有操作提供了自动求导机制。它是一个在运行时定义(define-by-run）的框架，这意味着反向传播是根据代码如何运行来决定的，并且每次迭代可以是不同的。**

## 张量

torch.Tensor 是这个包的核心类。如果设置它的属性 .requires_grad 为 True，那么它将会追踪对于该张量的所有操作。当完成计算后可以通过调用 .backward()，来自动计算所有的梯度。这个张量的所有梯度将会自动累加到.grad属性。要阻止一个张量被跟踪历史，可以调用 .detach() 方法将其与计算历史分离，并阻止它未来的计算记录被跟踪。

为了防止跟踪历史记录(和使用内存），可以将代码块包装在 with torch.no_grad(): 中。在评估模型时特别有用，因为模型可能具有 requires_grad = True 的可训练的参数，但是我们不需要在此过程中对他们进行梯度计算。

还有一个类对于autograd的实现非常重要：Function。

Tensor 和 Function 互相连接生成了一个无圈图(acyclic graph)，它编码了完整的计算历史。每个张量都有一个 .grad_fn 属性，该属性引用了创建 Tensor 自身的Function(除非这个张量是用户手动创建的，即这个张量的 grad_fn 是 None )。

如果需要计算导数，可以在 Tensor 上调用 .backward()。如果 Tensor 是一个标量(即它包含一个元素的数据），则不需要为 backward() 指定任何参数，但是如果它有更多的元素，则需要指定一个 gradient 参数，该参数是形状匹配的张量。

In [1]:
import torch

# 创建一个张量并设置requeires_grad=True用来追踪其计算历史
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [4]:
# 对此张量做一次运算
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
# y是计算结果，所以它具有grad_fn属性
print(y.grad_fn)

In [5]:
# 对y进行更多的操作
z = y * y * 3
out = z.mean()
print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


**.requires\_grad\_()原地改变现有张量的requires_grad标志。如果没有显示指定的化，该标志默认为False**

In [9]:
# 创建一个2*2的随机张量
a = torch.randn(2, 2)
# 对此张量做一次运算
a = ((a * 3)/(a - 1))
# 因为默认requries_grad为False，所以不会记录张量的计算行为
# 没有grad_fn属性
print(a)
print(a.grad_fn)
print(a.requires_grad)

# 使用requires_grad_()方法原地更改grad_fn属性为True
# 再次进行计算
a.requires_grad_(True)
b = (a * a).sum()
print(a)
print(b, b.grad_fn)

tensor([[-15.7307, -13.7455],
        [329.3542,   1.3176]])
None
False
tensor([[-15.7307, -13.7455],
        [329.3542,   1.3176]], requires_grad=True)
tensor(108912.3125, grad_fn=<SumBackward0>) <SumBackward0 object at 0x7f8785db5700>


## 梯度

**现在开始进行反向传播，因为 out 是一个标量，因此 out.backward() 和 out.backward(torch.tensor(1.)) 等价。**

In [11]:
# 现在out值是tensor(27., grad_fn=<MeanBackward0>)
# 由z.mean()计算得出，是一个标量，所以反响传播和out.backward(torch.tensor(1.))等价
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


**在数学上，如果y=f(x) (y, x均为向量)，那么y相对与x的梯度是一个雅可比矩阵**

以下是一个雅可比向量的例子

In [16]:
x = torch.randn(3, requires_grad=True)
y = x * 2
print(y)
print(y.data)
print(y.data.norm())
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([0.7679, 1.7053, 0.2041], grad_fn=<MulBackward0>)
tensor([0.7679, 1.7053, 0.2041])
tensor(1.8813)
tensor([ 786.3552, 1746.2151,  209.0412], grad_fn=<MulBackward0>)


这种情况下，y不再是标量。torch.autograd不能直接计算完整的雅可比矩阵，但是我们想要雅可比向量积，

只需要将这个向量作为参数传给backward。

In [17]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


**可以将代码块包装在with torch.no_grad():中，来组织autograd跟踪设置了requires_grad=True的张量的历史记录**

In [20]:
print(x.requires_grad)
print((x**2).requires_grad)
with torch.no_grad():
    print((x**2).requires_grad)

True
True
False
